## Testing String Manipulation and Functions

In [20]:
from datetime import datetime
import string

class split_converter:
    def split_to_lap(splits:list):
        ''' Take an input list of all the race splits and return list of laps '''

        splits = split_converter.fix_empty_splits(splits)
        conversion = [split_converter.interpret_input(y) - split_converter.interpret_input(x) for x,y in zip(splits,splits[1:])]
        formatted_conversion = [str(x) for x in conversion]

        return formatted_conversion

    def interpret_input(input:str):
        ''' Convert string to useable datetime value '''
        time_val = datetime.strptime(input, '%M:%S.%f')
        return time_val
    
    def fix_empty_splits(raw_input:list):
        if not any(bool(i) for i in raw_input):
            return raw_input
        
        

test = ('1:23.45', '2:40.43', '3:50.81', '3:59.49') + ('3:50.46', '3:50.44', '3:50.41', '3:50.43')

test2 = split_converter.interpret_input('1:23.45')
test3 = split_converter.interpret_input('2:34.56')
test4 = ['1:23.45', '2:34.56']

test5 = ['1:02.95', '0:52.90', '', '', '', '', '', '']

class time_addition:

    def add_time(str_list:list or tuple):
        minutes = sum([int(time[0]) for time in str_list])
        seconds = sum([int(sec[2:4]) for sec in str_list])
        partial_seconds = sum([int(hundredth[-2:]) for hundredth in str_list])

        minutes += seconds // 60
        remaining_seconds = seconds % 60
        seconds += partial_seconds // 100
        remaining_partial_seconds = partial_seconds % 100

        return f'{minutes}:{remaining_seconds:02}.{remaining_partial_seconds}'
    
[i for i in test5 if i != ''] == [i for i in test5 if i]

True

## Replacing Tables in the Database

In [9]:
from backup_database import db_backupper
import sqlite3

db_name = db_backupper().db_name

counter = 1

running_db = sqlite3.connect(db_name)
cursor = running_db.cursor()

for _ in range(2):
    if counter % 2 != 0:
        cursor.execute('''DROP TABLE relays''')
        print('dropped the table')
        print(counter)
        counter += 1
    else:
        cursor.execute('''
                CREATE TABLE IF NOT EXISTS relays (
                        meet_name TEXT,
                        date DATE,
                        event TEXT,
                        sex TEXT,
                        leg_1 TEXT,
                        leg_2 TEXT,
                        leg_3 TEXT,
                        leg_4 TEXT,
                        time TIME,
                        PRIMARY KEY (meet_name, date, event)
                    )
                   ''')
        print('added the table')
        print(counter)
        counter += 1

running_db.commit()
running_db.close()


added the table
2


## Troubleshoot time_math

In [21]:
from time_math import time_addition

time_addition.add_time(test4)

'3:57.1'

## Troubleshoot split_conversions

In [15]:
from split_conversions import split_converter

test6 = ['1:02.95', '2:52.87', '', '', '', '', '', '']
test7 = ['2:05.12', '4:05.12', '6:02.13', '8:06.23', '10:12.85', '12:34.56', '14:56.42', '17:00.29']


temp1 = [split_converter.interpret_input(y) - split_converter.interpret_input(x) for x,y in zip(test7,test7[1:])]
[str(i)[3:-4] if len(str(i)) > 7 else str(i)[3:] + '.00' for i in temp1] 


['2:00.00', '1:57.01', '2:04.10', '2:06.62', '2:21.71', '2:21.86', '2:03.87']

## Meet name db query

In [8]:
from backup_database import db_backupper
from database_operations import db_ops

db_name = db_backupper.db_name
db_ops.retrieve_meets(db_name)

['ahsaa 6a section 3 sectional meet',
 'ahsaa 6a state meet',
 'homewood invitational',
 'jaguar invitational',
 'king of the mountain invitational',
 'king of the west invitational',
 'mountain brook invitational',
 'northridge black and blue invitational',
 'paul w. bryant high school invitational',
 'southern mississippi high school invitational',
 'spain park invitational',
 'tuscaloosa/western alabama city/county championships']